In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = trp
band = [7]
commitnumber = de5d43e
molecule = {'o3': 'atmpro'}
tsfc = 300


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o o3 band5
  atmpro = mls
  band = 5
  commitnumber = a06b618
  conc = None
  dv = 0.001
  klin = 2e-20
  molecule = o3
  ng_adju = [0, 0]
  ng_refs = [2, 5]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (50, 250)]
  tsfc = 294
  vmax = 1100
  vmin = 980
  w_diffuse = [(1.6, 1.75), (1.55, 1.66, 1.7, 1.75, 1.8)]
  wgt = [(0.35, 0.6), (0.5, 0.55, 0.7, 0.9, 1.0)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0 -0.037163                  NaN       NaN
0.000312      NaN       NaN                  1.0 -0.029608
0.000750      2.0 -0.041619                  2.0 -0.033203
0.001052      3.0 -0.050184                  3.0 -0.040195
0.001476      4.0 -0.058309                  4.0 -0.046878
0.002070      5.0 -0.065995                  5.0 -0.053262
0.002904      6.0 -0.073232                  6.0 -0.059336
0.004074      7.0 -0.080012                  7.0 -0.065100
0.005714      8.0 -0.086320                  8.0 -0.070527
0.008015      9.0 -0.092154                  9.0 -0.075621
0.011243     10.0 -0.097618                 10.0 -0.080384
0.015771     11.0 -0.102794                 11.0 -0.084811
0.022122     12.0 -0.107767                 12.0 -0.088913
0.031031     13.0 -0.112700                 13.0 -0.092712
0.043528     14.0 -0.116676                 14.0 -0.095460
0.061057     15.0 -0.113553                 15.0 -0.093671
0.085645     16.0 -0.082259                 16.0 -0.071394
0.120136     17.0 -0.001398                 17.0 -0.008911
0.168516     18.0  0.136330                 18.0  0.104173
0.236378     19.0  0.340785                 19.0  0.280477
0.331549     20.0  0.622691                 20.0  0.534060
0.465100     21.0  0.995879                 21.0  0.879563
0.652400     22.0  1.431608                 22.0  1.289155
0.915100     23.0  1.703992                 23.0  1.584698
1.283650     24.0  1.691217                 24.0  1.637474
1.800600     25.0  1.523234                 25.0  1.497335
2.525700     26.0  1.209418                 26.0  1.170373
3.542800     27.0  0.858871                 27.0  0.818869
4.969550     28.0  0.574856                 28.0  0.552301
6.970850     29.0  0.367928                 29.0  0.369279
9.778100     30.0  0.168928                 30.0  0.188236
13.715850    31.0 -0.069323                 31.0 -0.052498
19.239350    32.0 -0.291495                 32.0 -0.289603
26.987250    33.0 -0.471295                 33.0 -0.491440
37.855300    34.0 -0.565125                 34.0 -0.618425
53.100050    35.0 -0.528503                 35.0 -0.598241
73.887500    36.0 -0.353656                 36.0 -0.389209
97.662500    37.0 -0.176670                 37.0 -0.184282
121.437500   38.0 -0.111453                 38.0 -0.113035
145.212500   39.0 -0.091896                 39.0 -0.092211
168.987500   40.0 -0.077819                 40.0 -0.077299
192.762500   41.0 -0.066675                 41.0 -0.065602
216.537500   42.0 -0.056188                 42.0 -0.054806
240.312500   43.0 -0.047134                 43.0 -0.045651
264.087500   44.0 -0.039500                 44.0 -0.038034
287.862500   45.0 -0.033213                 45.0 -0.031824
311.637500   46.0 -0.028126                 46.0 -0.026830
335.412500   47.0 -0.023761                 47.0 -0.022578
359.187500   48.0 -0.019799                 48.0 -0.018742
382.962500   49.0 -0.016298                 49.0 -0.015368
406.737500   50.0 -0.013210                 50.0 -0.012402
430.512500   51.0 -0.010311                 51.0 -0.009623
454.287500   52.0 -0.007532                 52.0 -0.006964
478.062500   53.0 -0.004862                 53.0 -0.004412
501.837500   54.0 -0.002293                 54.0 -0.001960
525.612500   55.0  0.000182                 55.0  0.000392
549.387500   56.0  0.002569                 56.0  0.002659
573.162500   57.0  0.004876                 57.0  0.004842
596.937500   58.0  0.007108                 58.0  0.006948
620.712500   59.0  0.009258                 59.0  0.008971
644.487500   60.0  0.011387                 60.0  0.010974
668.262500   61.0  0.013497                 61.0  0.012966
692.037500   62.0  0.015449                 62.0  0.014821
715.812500   63.0  0.017112                 63.0  0.016427
739.587500   64.0  0.018425                 64.0  0.017730
76

# Fluxes by Layer

CRD                          CLIRAD            \
                         flug      fldg      fnetg       flug      fldg   
pressure     level                                                        
0.000000e+00 1     -27.011626  0.000000 -27.011626        NaN       NaN   
1.000000e-08 1            NaN       NaN        NaN -26.764956  0.000002   
6.244000e-04 2     -27.011630  0.000002 -27.011628 -26.764959  0.000003   
8.759000e-04 3     -27.011632  0.000002 -27.011630 -26.764961  0.000004   
1.228600e-03 4     -27.011635  0.000004 -27.011632 -26.764963  0.000004   
1.723400e-03 5     -27.011641  0.000006 -27.011635 -26.764968  0.000006   
2.417400e-03 6     -27.011650  0.000010 -27.011641 -26.764975  0.000009   
3.390900e-03 7     -27.011665  0.000016 -27.011649 -26.764986  0.000013   
4.756500e-03 8     -27.011687  0.000025 -27.011662 -26.765003  0.000020   
6.672000e-03 9     -27.011722  0.000041 -27.011682 -26.765030  0.000031   
9.358900e-03 10    -27.011776  0.000065 -27.011711 -26.765071  0.000048   
1.312780e-02 11    -27.011857  0.000103 -27.011755 -26.765134  0.000075   
1.841450e-02 12    -27.011980  0.000161 -27.011819 -26.765229  0.000116   
2.583020e-02 13    -27.012164  0.000251 -27.011914 -26.765371  0.000180   
3.623230e-02 14    -27.012440  0.000387 -27.012052 -26.765584  0.000279   
5.082340e-02 15    -27.012850  0.000596 -27.012254 -26.765900  0.000430   
7.129060e-02 16    -27.013490  0.000960 -27.012529 -26.766393  0.000696   
1.000000e-01 17    -27.014523  0.001714 -27.012809 -26.767195  0.001255   
1.402710e-01 18    -27.016104  0.003288 -27.012816 -26.768429  0.002447   
1.967600e-01 19    -27.018329  0.006425 -27.011904 -26.770166  0.004881   
2.759970e-01 20    -27.021136  0.012431 -27.008705 -26.772332  0.009680   
3.871000e-01 21    -27.024025  0.023515 -27.000510 -26.774466  0.018843   
5.431000e-01 22    -27.025414  0.043308 -26.982107 -26.775191  0.035823   
7.617000e-01 23    -27.021750  0.076714 -26.945035 -26.771586  0.065605   
1.068500e+00 24    -27.010755  0.127647 -26.883108 -26.761733  0.113350   
1.498800e+00 25    -26.992825  0.195922 -26.796903 -26.745867  0.180958   
2.102400e+00 26    -26.967437  0.279446 -26.687991 -26.723725  0.265887   
2.949000e+00 27    -26.938745  0.372042 -26.566703 -26.699486  0.359032   
4.136600e+00 28    -26.914296  0.468418 -26.445878 -26.680694  0.455450   
5.802500e+00 29    -26.903156  0.570719 -26.332437 -26.676525  0.560283   
8.139200e+00 30    -26.920640  0.690045 -26.230595 -26.702755  0.688739   
1.141700e+01 31    -27.000730  0.835726 -26.165004 -26.793730  0.852810   
1.601470e+01 32    -27.192647  0.989888 -26.202759 -26.998321  1.028806   
2.246400e+01 33    -27.550167  1.124715 -26.425452 -27.370378  1.179593   
3.151050e+01 34    -28.157167  1.226665 -26.930502 -28.006386  1.288908   
4.420010e+01 35    -29.060916  1.280934 -27.779982 -28.990206  1.343032   
6.200000e+01 36    -30.166662  1.272317 -28.894345 -30.241644  1.332934   
8.577500e+01 37    -31.126098  1.235745 -29.890352 -31.302094  1.297135   
1.095500e+02 38    -31.605923  1.218013 -30.387910 -31.803756  1.279747   
1.333250e+02 39    -31.917395  1.215598 -30.701797 -32.118380  1.275995   
1.571000e+02 40    -32.186420  1.225815 -30.960605 -32.385280  1.283173   
1.808750e+02 41    -32.424637  1.244871 -31.179766 -32.618131  1.298302   
2.046500e+02 42    -32.638237  1.270692 -31.367545 -32.824240  1.319638   
2.284250e+02 43    -32.827096  1.301309 -31.525787 -33.004444  1.345474   
2.522000e+02 44    -32.994006  1.335474 -31.658532 -33.162251  1.374702   
2.759750e+02 45    -33.142194  1.372418 -31.769776 -33.301277  1.406601   
2.997500e+02 46    -33.275145  1.411831 -31.863314 -33.425184  1.440873   
3.235250e+02 47    -33.396352  1.453825 -31.942527 -33.537477  1.477596   
3.473000e+02 48    -33.507822  1.498377 -32.009445 -33.640220  1.516746   
3.710750e+02 49    -33.610428  1.545222 -32.065206 -33.734350  1.558088   
3.948500e+02 50    -33.705581  1.594475 -32.111107 

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')